In [522]:
from gensim import corpora, models, similarities, utils
import numpy as np
import pandas as pd
import nltk
import cPickle as pickle
import re

In [523]:
NWORDS_SHOWN = 15
SELECTED = [1,2,4,8,11,14,16,25]#topics to focus on

lda = pickle.load(open('lda_ortho_30t.p','rb'))

In [524]:
# texts = corpora.Dictionary.load('orthopedists.dict')
# print texts
#corpus = [dictionary.doc2bow(text) for text in texts]

In [541]:
# Make topic_word_df matrix:  Without filtering on POS
# Pull words lists for selected topics

#Initialize

topic_word_df=pd.DataFrame()
str_list=[]
words = []

for sel_topic in SELECTED:
    topic = lda.show_topic(sel_topic, NWORDS_SHOWN)#type list
    str_list=pd.DataFrame([(tup[1].encode("utf-8").replace(',','')) for tup in topic]).transpose() 
    str_list.insert(0,'Topic',sel_topic)
    topic_word_df = topic_word_df.append([str_list],ignore_index=True)

topic_word_df.ix[7,3]='image'#replace a name with a later word in this 
#print topic_word_df


In [542]:
def filter_POS(str_list, pos_tup):
    # Remove less informative parts of speech from the topics, so word
    # RB = AVB; CD = Cardinal Number; PRP/PRP$ = personal pronoun; RB=ADVERB; IN = Preposition
    #print pos_tup,'\n'

    # dict of topic, word pairs 
    table_dict={}
    
    for c in str_list.columns:
        if pos_tup[c][1]!='CD' and pos_tup[c][1]!='PRP' and pos_tup[c][1]!='IN' and pos_tup[c][1]!='RB' and pos_tup[c][1]!='CC':
        #if pos_tup[c][1]== [stop for stop in pow_stop]:
            table_dict[c+1]=str_list[c][0]#set key +1 because topics start at 1 not 0
            
    #pow_stop = ['CD','PRP','IN','RB']#tried to clean if statement with loop didn't work
    return table_dict

In [543]:
# CREATE FINAL DF of FILTERED TOP-WORD MAPPINGS FOR TABLE ON WEBSITE

# initialize
Ftopic_word_df=pd.DataFrame()# FINAL FILTERED TOP-WORD MAPPING DATAFRAME FOR TABLE ON WEBSITE
str_list=[]
frames = []# list of "topic-word map" dataframes for each topic

# loop over topics to get topic-word tuples
for sel_topic in selected:
    topic = lda.show_topic(sel_topic, NWORDS_SHOWN)#type list
    str_list=pd.DataFrame([(tup[1].encode("utf-8")) for tup in topic]).transpose() 
    #print(str_list)
    
    # Get Parts of Speech for that selected topic's words > pos DF
    words = str_list.loc[0,:]
    pos_tup = nltk.pos_tag(words)
    # for debugging
    if sel_topic==1:
        pos = pd.DataFrame([t for t in pos_tup]).transpose()
        print sel_topic, pos,'/n'

    # Make a dict & DF that filters the topic-word tuples by POS
    table_dict = filter_POS(str_list, pos_tup)
    topics_filtered_df = pd.DataFrame([t for t in table_dict.values()])
    topics_filtered_df = topics_filtered_df.transpose()
    topics_filtered_df.index = np.arange(0,topics_filtered_df.shape[0])
    topics_filtered_df.insert(0,'Topic #',sel_topic)
    
    #topics_filtered_df.insert(-1,'POS',pos_tup[:][1])
    #print topics_filtered_df.head(5)
    row_idx = selected.index(sel_topic)
    frames.insert(row_idx, topics_filtered_df)

Ftopic_word_df = pd.concat(frames)

print Ftopic_word_df.head(5)

1       0     1     2    3          4           5    6         7      8   9   \
0  knees  knee  pain  hip  surgeries  orthopedic  one  surgeons  great   &   
1    NNS   VBP    NN   NN        NNS          JJ   CD       NNS     JJ  CC   

         10    11    12    13    14  
0  patellas  time  left  i've  life  
1       NNS    NN   VBN    JJ    NN   /n
             0        1     2      3          4           5         6      7  \
0        knees     knee  pain    hip  surgeries  orthopedic  surgeons  great   
0          hip     pain  knee   time        i'm        care      good    guy   
0         time     care  best  staff    treated      office  shoulder   good   
0         knee     pain  time  years    running  orthopedic       saw   work   
0  appointment  minutes  time  staff    waiting         day      told    me,   

             8           9      10       11      12   13  Topic #  
0     patellas        time    left     i've    life  NaN        1  
0       labrum         day   

In [544]:
topic_word_df.head(5)

,Topic,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,knees,knee,pain,hip,surgeries,orthopedic,one,surgeons,great,&,patellas,time,left,i've,life
1,2,hip,pain,knee,time,right,back,i'm,care,good,guy,labrum,day,since,hips,left
2,4,time,really,care,like,best,staff,treated,office,shoulder,good,appointment,orthopedic,always,patient,great
3,8,knee,pain,much,time,years,running,orthopedic,saw,work,good,many,without,best,even,team
4,11,appointment,minutes,time,staff,waiting,day,told,one,me,late,first,say,luke,office,mri


In [545]:
Ftopic_word_df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,Topic #
0,knees,knee,pain,hip,surgeries,orthopedic,surgeons,great,patellas,time,left,i've,life,NaN,1
0,hip,pain,knee,time,i'm,care,good,guy,labrum,day,hips,left,NaN,NaN,2
0,time,care,best,staff,treated,office,shoulder,good,appointment,orthopedic,always,patient,great,NaN,4
0,knee,pain,time,years,running,orthopedic,saw,work,good,many,best,team,NaN,NaN,8
0,appointment,minutes,time,staff,waiting,day,told,"me,",late,first,say,luke,office,mri,11


In [488]:
### How to pull words out of dict
#print table_dict.items()
#mywords = table_dict.values()
#print mywords

In [489]:
# save word df

pickle_out = open('table_dict.p', 'wb')
pickle.dump(table_dict, pickle_out)

# pickle_out = open('topic_word_df.p', 'wb')
# pickle.dump(topic_word_df, pickle_out)

# pickle_out = open('Ftopic_word_df.p', 'wb')
# pickle.dump(Ftopic_word_df, pickle_out)

In [320]:
import csv

def write_csv(filename, words):
    with open(filename,'w',newline='') as csvfile:
        wr = csv.writer(csvfile, delimiter=',',
                        quotechar='|',quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
        #wr.writerow(table_dict.keys())
        wr.writerows(words) 
    print('\nCSV file saved.\n')
    
write_csv('table_dict.csv', mywords)

TypeError: 'newline' is an invalid keyword argument for this function

In [540]:
a = np.array([[0,1,2],[3,4,5]])
print a
temp = pd.DataFrame(a)
print '\nDataframe = \n',temp

temp.index = [3,4]
print '\nDataframe = \n',temp

[[0 1 2]
 [3 4 5]]

Dataframe = 
   0  1  2
0  0  1  2
1  3  4  5

Dataframe = 
   0  1  2
3  0  1  2
4  3  4  5
